In [2]:
import numpy as np
import mysql.connector as connection
import pandas as pd
from scipy.stats import zscore
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score, cross_val_predict,ShuffleSplit
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
from joblib import dump, load
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as msa

import category_encoders as ce
import catboost as cb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [5]:
años = [2015,2016,2017,2018,2019,2020,2021]
lista_db = []
try:
    mydb = connection.connect(host="localhost", database = 'multilab',user="root", passwd="admin",use_pure=True)
    for año in años:
        query = f"Select * from muestra_{año};"
        lista_db.append(pd.read_sql(query,mydb))
    mydb.close() #close the connection
except Exception as e:
    mydb.close()
    print(str(e))

In [6]:
lista_db[0].columns

Index(['codigo', 'orden', 'numero_muestra', 'referencia', 'clase_analisis',
       'tipo_analisis', 'fecha_analisis', 'proyecto', 'ph', 'humedad',
       'residuo_seco', 'ce', 'n', 'ndet', 'mo', 'k', 'ca', 'mg', 'na', 'al',
       'cic', 'p', 'fe', 'mn', 'zn', 'cu', 's', 'b', 'ar', 'l', 'a',
       'reteagua', 'cooxid', 'densidad', 'clasificacion', 'textura'],
      dtype='object')

In [70]:
db_filtrada = []
for db in lista_db:
    db_filtrada.append(db[[ 'ph', 'mo', 'k', 'ca', 'mg']])

In [71]:
df = pd.concat(db_filtrada)

In [72]:
df = df.dropna()

In [73]:
cols = df.columns.values
df = df.replace(",",".",regex=True)

In [74]:
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce', axis=1)
df = df.dropna()

In [75]:


X = df[['ph',  'k', 'ca', 'mg',
    'ar', 'l', 'a']]
y = df['mo']

sc = StandardScaler()
train_cbe = sc.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(train_cbe, y, test_size=0.2, random_state=42)

In [76]:
import xgboost as xg

In [77]:
model =   xg.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=2, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.4603, gamma=0.0468,
             gpu_id=-1, importance_type='gain', interaction_constraints=None,
             learning_rate=0.05, max_delta_step=0, max_depth=2,
             min_child_weight=1.7817, missing=nan, monotone_constraints=None,
             n_estimators=2200, n_jobs=-1, nthread=-1, num_parallel_tree=1,
             random_state=7, reg_alpha=0.464, reg_lambda=0.8571,
             scale_pos_weight=1, silent=1, subsample=0.5213, tree_method=None,
             validate_parameters=False, verbosity=None)

In [78]:
from sklearn.metrics import mean_absolute_error
pred = model.predict(X_test)
rmse = (mean_absolute_error(y_test, abs(pred)))
r2 = r2_score(y_test, pred)
print("Testing performance")
print("MAE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))

Testing performance
MAE: 3.02
R2: 0.23


In [69]:
model.feature_importances_

array([0.06380271, 0.06094562, 0.11930764, 0.10679746, 0.07866587,
       0.05219775, 0.07208496, 0.09057162, 0.05703782, 0.04917523,
       0.05367559, 0.07154549, 0.05182622, 0.0723661 ], dtype=float32)